In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tree import DecisionTreeClassifier

In [69]:
# Load the data
data = pd.read_csv('./data/breast_cancer.csv',index_col=0)
data.head() 

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [70]:
data.shape

(569, 31)

In [71]:
### Creating Rnadom Forest Algorithm ###
from collections import Counter

class RandomForest:
    def __init__(self,n_trees=10,max_depth=10,min_samples_split=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split= min_samples_split
        self.trees = []
    
    def fit(self,X,y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            X_sample,y_sample = self._bootstrap_samples(X,y)
            tree.fit(X_sample,y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self,X,y):
        X = np.array(X)
        y = np.array(y)
        n_samples = X.shape[0]
        idx = np.random.choice(n_samples,n_samples,replace=True)
        return X[idx],y[idx]

    def _most_common(self,y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self,X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        trees_pred = np.swapaxes(predictions,0,1)
        return np.array([self._most_common(pred) for pred in trees_pred])

    def metrics(self,y_true,y_pred):
        acc = accuracy_score(y_true,y_pred)
        pre = precision_score(y_true,y_pred)
        f1 = f1_score(y_true,y_pred)
        confusion = confusion_matrix(y_true,y_pred)

        print(f'Accuracy: {acc*100:.2f}%')
        print(f'Precision: {pre*100:.2f}%')
        print(f'F1 Sore: {f1*100:.4f}%')
        print('Confusion Matrix: \n', confusion)

In [72]:
# Data split
X = data[data.columns[:-1]].values
y = data['target']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=8)

In [75]:
# Fiting the model
rf = RandomForest(
    n_trees=50,
    max_depth=10,
    min_samples_split=2
)
rf.fit(X_train,y_train)

In [76]:
pred=rf.predict(X_test)
rf.metrics(y_test,pred)

Accuracy: 97.90%
Precision: 97.67%
F1 Sore: 98.2456%
Confusion Matrix: 
 [[56  2]
 [ 1 84]]
